## Q1. Getting the embeddings model


In [1]:
from sentence_transformers import SentenceTransformer

/home/luiscs/projects/github/llm-bootcamp/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(model_name)

In [8]:
user_question = "I just discovered the course. Can I still join it?"

In [9]:
embedding_model.encode(user_question)[0]

0.078222655

### Prepare the documents

In [10]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [11]:
len(documents)

948

In [12]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [13]:
documents_ml = [ d for d in documents if d['course'] == 'machine-learning-zoomcamp']

In [14]:
len(documents_ml)

375

## Q2. Creating the embeddings


In [15]:
import numpy as np

In [16]:
embeddings = list()

for doc in documents_ml:
    qa_text = f'{doc["question"]} {doc["text"]}'
    qa_emb = embedding_model.encode(qa_text)
    embeddings.append(qa_emb)
   
X = np.array(embeddings) 

In [17]:
X.shape

(375, 768)

## Q3. Search

In [18]:
v = embedding_model.encode(user_question)
scores = X.dot(v)

In [20]:
np.max(scores)

0.6506573

### Vector search

In [21]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

In [29]:
search_engine = VectorSearchEngine(documents=documents_ml, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

## Q4. Hit-rate for our search engine


In [30]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [33]:
ground_truth

[{'question': 'Where can I sign up for the course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Can you provide a link to sign up?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Is there an FAQ for this Machine Learning course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Does this course have a GitHub repository for the sign-up link?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'How can I structure my questions and answers for the course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Are the course videos live or pre-recorded?',
  'course': 'machine-learning-zoomcamp',
  'document': '39fda9f0'},
 {'question': 'When can I start watching the course videos?',
  'course': 'machine-learning-zoomcamp',
  'document': '39fda9f0'},
 {'question': 'Are the live office hours sessions recorded?',
  'cours

In [26]:
from tqdm.auto import tqdm

In [42]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [43]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [44]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [45]:
def text_vector_knn(q):
    question = q['question']
    course = q['course']
    v_q = embedding_model.encode(question)

    return search_engine.search(v_q, num_results=5)

In [46]:
evaluate(ground_truth, text_vector_knn)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1830/1830 [04:21<00:00,  7.00it/s]


{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}

## Q5. Indexing with Elasticsearch

In [47]:
len(embedding_model.encode(user_question))

768

In [49]:
embeddings[0]

array([ 8.80590528e-02,  1.55935483e-02,  7.92558193e-02,  2.52757873e-02,
        7.55765066e-02, -3.90596949e-02, -4.13813852e-02,  2.52918135e-02,
        2.43241284e-02,  3.62585997e-03, -7.28285545e-03, -3.28751542e-02,
        6.12956472e-02, -5.71101010e-02,  1.16774784e-02, -1.79442130e-02,
        4.49206047e-02, -5.41605875e-02, -1.92261033e-03,  1.48329930e-02,
        7.91360065e-03, -3.43127362e-02,  1.21480487e-02,  1.27185713e-02,
       -1.60092711e-02,  7.12137856e-03,  1.58389509e-02, -3.27488902e-04,
        3.01296543e-03, -4.58918400e-02, -1.34775438e-03,  2.01149546e-02,
        2.33724471e-02, -4.28200746e-03, -3.09384037e-02,  1.94302748e-03,
       -1.88219752e-02,  5.35160303e-03,  1.29352352e-02, -3.23957913e-02,
        3.10556218e-02,  5.12981648e-03, -3.24394932e-04, -3.15541998e-02,
        1.93377174e-02, -6.38601556e-02, -3.05381045e-02, -6.27884790e-02,
       -1.74859781e-02,  3.52949575e-02, -2.98071913e-02,  7.45346099e-02,
       -2.06183232e-02, -

In [50]:
documents_ml[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [51]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            }
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [52]:

i = 0
for doc in tqdm(documents_ml):
    doc['question_vector'] = embeddings[i]
    es_client.index(index=index_name, document=doc)
    i += 1

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [00:03<00:00, 117.46it/s]


In [53]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [64]:
elastic_search_knn('question_vector', v, 'machine-learning-zoomcamp')

[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'I just joined. What should I do next? How can I access course materials?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the vid

## Q6. Hit-rate for Elasticsearch

In [65]:
def text_vector_knn_elastic(q):
    question = q['question']
    course = q['course']
    v_q = embedding_model.encode(question)

    return elastic_search_knn('question_vector', v_q, course)

In [66]:
evaluate(ground_truth, text_vector_knn_elastic)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1830/1830 [01:09<00:00, 26.36it/s]


{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}